<a href="https://colab.research.google.com/github/devinlin2022/Streamlit/blob/main/EU_Commodity_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New

In [1]:
!pip install pygsheets
!pip install google-colab-selenium
!pip install retrying
!pip install PyMuPDF==1.24.9
!pip install fitz==1.16.14

import base64
import io
import time
import pandas as pd
import requests
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import fitz
# from pdf2image import convert_from_path
# from fitz import open as fitz_open
import fitz
import pygsheets
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default
import gspread
import re
import json

# Authenticate user
auth.authenticate_user()
creds, _ = default()

def save_pdf(driver, path):
    settings = {
        "landscape": False,
        "displayHeaderFooter": False,
        "printBackground": True,
        "preferCSSPageSize": True
    }
    result = driver.execute_cdp_cmd("Page.printToPDF", settings)
    pdf_data = base64.b64decode(result['data'])
    with open(path, 'wb') as f:
        f.write(pdf_data)


# def save_as_jpg(driver, path):

#     screenshot = driver.get_screenshot_as_png()

#     image = Image.open(io.BytesIO(screenshot))

#     image = image.convert('RGB')
#     image.save(path, 'JPEG')

def fetch_data(link):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)
    driver.get(link)

    wait = WebDriverWait(driver, 30)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#login-button')))

    driver.execute_script(f'document.querySelector("#username-input").value = "agnieszka_wierzyk@colpal.com"')
    driver.execute_script(f'document.querySelector("#password-input").value = "Commodity@2025"')
    driver.execute_script(f'document.querySelector("#login-button").click()')

    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#continue-login-button')))
    time.sleep(10)

    try:
        driver.execute_script(f'document.querySelector("#continue-login-button").click()')
    except:
        pass

    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,
        '#content > div > div > div > div > div.Zoomstyle__BodyContainer-LbgNq.fhHJpQ > div.Zoomstyle__Section-hqZqfX.jKLgrv > div.Largestyle__DisplayWrapperLarge-iWzxqM.hISDst > div.Largestyle__DisplayItem-vzpFY.fbUftf > div > div:nth-child(2) > div > div > div.PriceDeltastyle__DeltaContainer-jdFEoE.dtfcmD > div.Textstyles__Heading1Blue-gtxuIB.dzShK'
    )))

    save_pdf(driver, "webpage.pdf")

    element_scripts = [
        """return document.querySelector("#content > div > div > div > div > div.Zoomstyle__BodyContainer-LbgNq.fhHJpQ > div.Zoomstyle__Section-hqZqfX.jKLgrv > div.Largestyle__DisplayWrapperLarge-iWzxqM.hISDst > div.Largestyle__DisplayItem-vzpFY.fbUftf > div > div:nth-child(1) > div > div > div.PriceDeltastyle__DeltaContainer-jdFEoE.dtfcmD > div.Textstyles__Heading1Blue-gtxuIB.dzShK").textContent;""",
        """return document.querySelector("#content > div > div > div > div > div.Zoomstyle__BodyContainer-LbgNq.fhHJpQ > div.Zoomstyle__Section-hqZqfX.jKLgrv > div.Largestyle__DisplayWrapperLarge-iWzxqM.hISDst > div.Largestyle__DisplayItem-vzpFY.fbUftf > div > div:nth-child(2) > div > div > div.PriceDeltastyle__DeltaContainer-jdFEoE.dtfcmD > div.Textstyles__Heading1Blue-gtxuIB.dzShK").textContent;""",
        """return document.querySelector("#content > div > div > div > div > div.Zoomstyle__BodyContainer-LbgNq.fhHJpQ > div.Zoomstyle__Section-hqZqfX.jKLgrv > div.Largestyle__DisplayWrapperLarge-iWzxqM.hISDst > div.Largestyle__DisplayItem-vzpFY.fbUftf > div > div:nth-child(3) > div > div > div.PriceDeltastyle__DeltaContainer-jdFEoE.dtfcmD > div.Textstyles__Heading1Blue-gtxuIB.dzShK").textContent;"""
    ]

    elements = [driver.execute_script(script) for script in element_scripts]

    date_element = driver.find_element(By.CSS_SELECTOR,
        '#content > div > div > div > div > div.Zoomstyle__BodyContainer-LbgNq.fhHJpQ > div.Zoomstyle__Section-hqZqfX.jKLgrv > div.Largestyle__DisplayWrapperLarge-iWzxqM.hISDst > div.Mainstyle__Group-ciNpsy.fYvNPb > div > div > div:nth-child(2) > div')
    date = date_element.text
    driver.quit()
    return elements, date

def upload_to_google_sheet(data, sheet_key, worksheet_name, row):
    try:
        gc = pygsheets.authorize(custom_credentials=creds)
        wb_key = gc.open_by_key(sheet_key)
        sheet = wb_key.worksheet_by_title(worksheet_name)

        if len(row) > 1 and row[1].strip() == "ICIS":
            # Convert the date to YYYY-MM-DD format
            original_date = data['Date'][0]
            try:
                formatted_date = datetime.strptime(original_date, '%d-%b-%y').strftime('%Y-%m-%d')
            except ValueError as e:
                print(f"Failed to parse date {original_date}: {e}")
                return

            # Spread the list across columns
            new_row = [formatted_date] + data['Commodity'][0]
        else:
            print(f"Unexpected value in column B: {row[1]}")
            return  # Exit if column B does not match

        all_values = sheet.get_all_values(returnas='matrix')
        last_non_empty_row = next((i for i, existing_row in reversed(list(enumerate(all_values))) if any(cell.strip() for cell in existing_row)), -1)

        # Offset by 1 to preserve header
        new_row_index = max(last_non_empty_row + 1, 1)
        if new_row_index >= sheet.rows:
            # Add more rows if needed
            sheet.add_rows(new_row_index - sheet.rows + 100)

        # Update the row with new data, starting from the second row
        sheet.update_row(new_row_index + 1, new_row)  # +1 to account for 0-based index and header row
        print(f"Row successfully added to {worksheet_name} at row {new_row_index + 1}.")
    except Exception as e:
        print(f"Failed to add row to {worksheet_name}: {e}")

def pdf_to_img(pdf_path, img_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)

    # Get the first page
    page = pdf_document[0]

    # Render page to an image
    pixmap = page.get_pixmap(dpi=300)  # Adjust DPI as necessary

    # Save the image
    pixmap.save(img_path)

# def pdf_to_img(pdf_path, img_path):
#     # Convert PDF to image and save the first page as an image
#     images = convert_from_path(pdf_path, dpi=300)
#     images[0].save(img_path, 'JPEG')

def save_as_jpg(driver, path):
    screenshot = driver.get_screenshot_as_png()
    image = Image.open(io.BytesIO(screenshot))
    image = image.convert('RGB')
    image.save(path, 'JPEG')

def save_pdf(driver, path):
    settings = {
        "landscape": False,
        "displayHeaderFooter": False,
        "printBackground": True,
        "preferCSSPageSize": True
    }
    result = driver.execute_cdp_cmd("Page.printToPDF", settings)
    pdf_data = base64.b64decode(result['data'])
    with open(path, 'wb') as f:
        f.write(pdf_data)
    time.sleep(3)
    pdf_to_img('/content/webpage.pdf', '/content/01.jpg')

def fetch_RISI_data(link, search_term):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)
    driver.get(link)

    wait = WebDriverWait(driver, 100)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#userEmail')))

    driver.execute_script(f'document.querySelector("#userEmail").value = "beata_zbrozek@colpal.com"')
    driver.execute_script(f'document.querySelector("#password").value = "Risi2025!"')
    driver.execute_script(f'document.querySelector("#login-button").click()')
    time.sleep(5)

    # Dynamic search term
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mat-mdc-chip-list-input-0')))
    input_element = driver.execute_script("return document.querySelector('#mat-mdc-chip-list-input-0')")

    input_element.clear()
    input_element.send_keys(search_term, Keys.ENTER)

    time.sleep(5)
    driver.execute_script(f'document.querySelector("body > fui-root-container > fui-root > fui-dashboard-container > fui-dashboard > main > header > fui-workspace-header-container > fui-dashboard-header-container > fui-dashboard-header > div.header-section.search.draggable > fui-search-bar > fui-dropdown-container > div > div.content > div > fui-global-search-dropdown > div > section.results-view.ng-star-inserted > div > fui-search-feed > ul > cdk-virtual-scroll-viewport > div.cdk-virtual-scroll-content-wrapper > li > fui-instrument-search-result-item > fui-search-result-item").click()')
    time.sleep(5)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#mat-tab-label-1-3")))
    driver.execute_script(f'document.querySelector("#mat-tab-label-1-3").click()')
    time.sleep(5)
    save_pdf(driver, '/content/webpage.pdf')
    driver.close()

    return driver

def update_google_sheet_from_csv(csv_path, sheet_id, sheet_name):
    # Load CSV with correct headers setup
    df = pd.read_csv(csv_path)

    df.columns = df.iloc[0]  # Promote first row to header
    df = df[1:]  # Remove the header from the data
    df.reset_index(drop=True, inplace=True)

    df[df.columns[0]] = pd.to_datetime(df[df.columns[0]], errors='coerce')
    df = df.dropna(subset=[df.columns[0]])

    # Convert dates explicitly to string in the desired format
    df[df.columns[0]] = df[df.columns[0]].dt.strftime('%Y-%m-%d')

    df = df.sort_values(by=df.columns[0])

    client = gspread.authorize(creds)
    sh = client.open_by_key(sheet_id)
    worksheet = sh.worksheet(sheet_name)

    existing_data = pd.DataFrame(worksheet.get_all_values())

    if not existing_data.empty:
        existing_data.columns = existing_data.iloc[0]
        existing_data = existing_data[1:]
        existing_data.reset_index(drop=True, inplace=True)

    new_rows = df.iloc[2:]

    if not existing_data.empty and df.columns[0] in existing_data:
        records_to_add = new_rows[
            ~new_rows[df.columns[0]].isin(existing_data[existing_data.columns[0]].astype('str'))]
    else:
        records_to_add = new_rows

    if not records_to_add.empty:
        set_with_dataframe(worksheet, records_to_add, include_index=False,
                           include_column_header=False, row=len(existing_data) + 2)
        print("New data appended to Google Sheets successfully.")
    else:
        print("No new data to append.")

class ImageService:
    def __init__(self, api_key):
        self.api_key = api_key

    def get_image_base64_string(self, image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    def send_post_request(self, url, json_content):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        response = requests.post(url, data=json_content, headers=headers)
        if 200 <= response.status_code < 300:
            return response.text
        else:
            raise Exception(f"Request failed: {response.status_code}, {response.text}")

    # def get_results(self, image_path):
    #     url = "https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions"
    #     encoded_image = self.get_image_base64_string(image_path)
    #     json_content = {
    #         "model": "qwen2-vl-7b-instruct",
    #         "messages": [
    #             {
    #                 "role": "user",
    #                 "content": [
    #                     {"type": "text", "text": "抓取图片中表格。"},
    #                     {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}}
    #                 ]
    #             }
    #         ]
    #     }
    #     json_content_str = json.dumps(json_content)
    #     result = self.send_post_request(url, json_content_str)
    #     return result

    def get_results(self, image_path):
        url = "https://dashscope.aliyuncs.com/compatible-model/v1/chat/completions"
        encoded_image = self.get_image_base64_string(image_path)
        json_content = {
            "model": "qwen-vl-plus",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "抓取图片中表格。"},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}}
                    ]
                }
            ]
        }
        json_content_str = json.dumps(json_content)
        result = self.send_post_request(url, json_content_str)
        return result

    def parse_table_and_save_to_csv(self, response_body, csv_path):
        data = json.loads(response_body)
        if 'choices' in data and len(data['choices']) > 0:
            message = data['choices'][0]['message']['content']
            table_lines = re.findall(r"\|.*\|", message)

            if table_lines:
                rows = []
                for line in table_lines[1:]:
                    columns = [col.strip() for col in line.split('|')[1:-1]]
                    rows.append(columns)

                header = [col.strip() for col in table_lines[0].split('|')[1:-1]]
                table_df = pd.DataFrame(rows, columns=header)
                table_df.to_csv(csv_path, index=False)
                print(f"Extracted table saved to {csv_path}")
            else:
                print("No table data found in response.")
        else:
            print("No table data found in response.")

def get_dynamic_parameters(sheet_id):
    client = gspread.authorize(creds)
    sh = client.open_by_key(sheet_id)
    worksheet = sh.worksheet("ALL_EU_AEA")

    data = worksheet.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])

    filtered_df = df[df['Tab Name'].str.startswith("RISI")]

    return filtered_df[['Address', 'Tab Name', 'Series code']].to_dict('records')

def risi_main():
    sheet_id = '1Qonj5yKwHVrxApUi7_N2CJtxj61rPfULXALrY4f8lPE'

    rows = get_dynamic_parameters(sheet_id)

    api_key = "sk-db6fa6efbf144521b0180da5e83b0a94"
    image_path = '/content/01.jpg'

    service = ImageService(api_key)

    for row in rows:
        link = row['Address']
        sheet_name = row['Tab Name']
        search_term = row['Series code']  # Dynamic search term from column E

        # Fetch data from the website using the dynamic search term
        fetch_RISI_data(link, search_term)
        csv_path = '/content/extracted_table.csv'
        try:
            result = service.get_results(image_path)
            print(result)
            service.parse_table_and_save_to_csv(result, csv_path)
        except Exception as e:
            print(f"An error occurred: {e}")

        # Update Google Sheet
        update_google_sheet_from_csv(csv_path, sheet_id, sheet_name)


def main(id, tab_name):
    gc = pygsheets.authorize(custom_credentials=creds)
    sh = gc.open_by_key(id)
    wks = sh.worksheet_by_title(tab_name)

    data = wks.get_all_values()

    sheet_key = '1Qonj5yKwHVrxApUi7_N2CJtxj61rPfULXALrY4f8lPE'
    worksheet_name_col_idx = 8  # Column I (0-based index is 8)
    link_col_idx = 5            # Column F (0-based index is 5)

    for row in data[1:]:  # Skip header row
        if len(row) <= max(worksheet_name_col_idx, link_col_idx):
            continue  # Skip if there are not enough columns

        worksheet_name = row[worksheet_name_col_idx]
        link = row[link_col_idx]

        if not worksheet_name or not link:
            continue  # Skip if any critical information is missing

        # Apply the constraints only for rows where column B is "ICIS"
        if len(row) > 1 and row[1].strip() == "ICIS":
            price, date = fetch_data(link)

            # Replace `break` with a switch to `risi_main()` if required
            if 'Element Not Found' in price or date == 'Date Not Found':
                print(f"Switching to RISI data processing for ICIS row with link: {link}")
                risi_main()  # Call risi_main instead of breaking
                continue  # Ensure loop continues after switching

        # Only proceed if valid data is found
        price_data = pd.DataFrame([[date, price]], columns=['Date', 'Commodity'])
        print(price_data)

        upload_to_google_sheet(price_data, sheet_key, worksheet_name, row)

if __name__ == "__main__":
    main('1Qonj5yKwHVrxApUi7_N2CJtxj61rPfULXALrY4f8lPE', 'ALL_EU_AEA')

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

ModuleNotFoundError: No module named 'tools'

# Run the code by click button on the left

In [4]:
!pip install pygsheets
!pip install google-colab-selenium
!pip install retrying

import time
import base64
import pandas as pd
from retrying import retry
from datetime import datetime
from google.colab import auth
from google.auth import default
from PIL import Image
import io
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pygsheets

# Authenticate user
auth.authenticate_user()
creds, _ = default()

def save_pdf(driver, path):
    settings = {
        "landscape": False,
        "displayHeaderFooter": False,
        "printBackground": True,
        "preferCSSPageSize": True
    }
    result = driver.execute_cdp_cmd("Page.printToPDF", settings)
    pdf_data = base64.b64decode(result['data'])
    with open(path, 'wb') as f:
        f.write(pdf_data)


# def save_as_jpg(driver, path):

#     screenshot = driver.get_screenshot_as_png()

#     image = Image.open(io.BytesIO(screenshot))

#     image = image.convert('RGB')
#     image.save(path, 'JPEG')

def fetch_data(link):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)
    driver.get(link)

    wait = WebDriverWait(driver, 50)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#login-button')))

    driver.execute_script(f'document.querySelector("#username-input").value = "agnieszka_wierzyk@colpal.com"')
    driver.execute_script(f'document.querySelector("#password-input").value = "Commodity@2025"')
    driver.execute_script(f'document.querySelector("#login-button").click()')

    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#continue-login-button')))
    time.sleep(10)

    try:
        driver.execute_script(f'document.querySelector("#continue-login-button").click()')
    except:
        pass

    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,
        '#content > div > div > div > div > div.Zoomstyle__BodyContainer-LbgNq.fhHJpQ > div.Zoomstyle__Section-hqZqfX.jKLgrv > div.Largestyle__DisplayWrapperLarge-iWzxqM.hISDst > div.Largestyle__DisplayItem-vzpFY.fbUftf > div > div:nth-child(2) > div > div > div.PriceDeltastyle__DeltaContainer-jdFEoE.dtfcmD > div.Textstyles__Heading1Blue-gtxuIB.dzShK'
    )))

    save_pdf(driver, "webpage.pdf")

    element_scripts = [
        """return document.querySelector("#content > div > div > div > div > div.Zoomstyle__BodyContainer-LbgNq.fhHJpQ > div.Zoomstyle__Section-hqZqfX.jKLgrv > div.Largestyle__DisplayWrapperLarge-iWzxqM.hISDst > div.Largestyle__DisplayItem-vzpFY.fbUftf > div > div:nth-child(1) > div > div > div.PriceDeltastyle__DeltaContainer-jdFEoE.dtfcmD > div.Textstyles__Heading1Blue-gtxuIB.dzShK").textContent;""",
        """return document.querySelector("#content > div > div > div > div > div.Zoomstyle__BodyContainer-LbgNq.fhHJpQ > div.Zoomstyle__Section-hqZqfX.jKLgrv > div.Largestyle__DisplayWrapperLarge-iWzxqM.hISDst > div.Largestyle__DisplayItem-vzpFY.fbUftf > div > div:nth-child(2) > div > div > div.PriceDeltastyle__DeltaContainer-jdFEoE.dtfcmD > div.Textstyles__Heading1Blue-gtxuIB.dzShK").textContent;""",
        """return document.querySelector("#content > div > div > div > div > div.Zoomstyle__BodyContainer-LbgNq.fhHJpQ > div.Zoomstyle__Section-hqZqfX.jKLgrv > div.Largestyle__DisplayWrapperLarge-iWzxqM.hISDst > div.Largestyle__DisplayItem-vzpFY.fbUftf > div > div:nth-child(3) > div > div > div.PriceDeltastyle__DeltaContainer-jdFEoE.dtfcmD > div.Textstyles__Heading1Blue-gtxuIB.dzShK").textContent;"""
    ]

    elements = [driver.execute_script(script) for script in element_scripts]

    date_element = driver.find_element(By.CSS_SELECTOR,
        '#content > div > div > div > div > div.Zoomstyle__BodyContainer-LbgNq.fhHJpQ > div.Zoomstyle__Section-hqZqfX.jKLgrv > div.Largestyle__DisplayWrapperLarge-iWzxqM.hISDst > div.Mainstyle__Group-ciNpsy.fYvNPb > div > div > div:nth-child(2) > div')
    date = date_element.text
    driver.quit()
    return elements, date

def upload_to_google_sheet(data, sheet_key, worksheet_name, row):
    try:
        gc = pygsheets.authorize(custom_credentials=creds)
        wb_key = gc.open_by_key(sheet_key)
        sheet = wb_key.worksheet_by_title(worksheet_name)

        if len(row) > 1 and row[1].strip() == "ICIS":
            # Convert the date to YYYY-MM-DD format
            original_date = data['Date'][0]
            try:
                formatted_date = datetime.strptime(original_date, '%d-%b-%y').strftime('%Y-%m-%d')
            except ValueError as e:
                print(f"Failed to parse date {original_date}: {e}")
                return

            # Spread the list across columns
            new_row = [formatted_date] + data['Commodity'][0]
        else:
            print(f"Unexpected value in column B: {row[1]}")
            return  # Exit if column B does not match

        all_values = sheet.get_all_values(returnas='matrix')
        last_non_empty_row = next((i for i, existing_row in reversed(list(enumerate(all_values))) if any(cell.strip() for cell in existing_row)), -1)

        # Offset by 1 to preserve header
        new_row_index = max(last_non_empty_row + 1, 1)
        if new_row_index >= sheet.rows:
            # Add more rows if needed
            sheet.add_rows(new_row_index - sheet.rows + 100)

        # Update the row with new data, starting from the second row
        sheet.update_row(new_row_index + 1, new_row)  # +1 to account for 0-based index and header row
        print(f"Row successfully added to {worksheet_name} at row {new_row_index + 1}.")
    except Exception as e:
        print(f"Failed to add row to {worksheet_name}: {e}")

def main(id, tab_name):
    gc = pygsheets.authorize(custom_credentials=creds)
    sh = gc.open_by_key(id)
    wks = sh.worksheet_by_title(tab_name)

    data = wks.get_all_values()

    sheet_key = '1Qonj5yKwHVrxApUi7_N2CJtxj61rPfULXALrY4f8lPE'
    worksheet_name_col_idx = 8  # Column I (0-based index is 8)
    link_col_idx = 5            # Column F (0-based index is 5)

    for row in data[1:]:  # Skip header row
        if len(row) <= max(worksheet_name_col_idx, link_col_idx):
            continue  # Skip if there are not enough columns

        worksheet_name = row[worksheet_name_col_idx]
        link = row[link_col_idx]

        if not worksheet_name or not link:
            continue  # Skip if any critical information is missing

        # Apply the constraints only for rows where column B is "ICIS"
        if len(row) > 1 and row[1].strip() == "ICIS":
            price, date = fetch_data(link)

            # Exit loop if 'Element Not Found' or 'Date Not Found' is encountered
            # And it's an "ICIS" row
            if 'Element Not Found' in price or date == 'Date Not Found':
                print(f"Stopping due to missing data for ICIS row with link: {link}")
                break  # Use return here if you want to completely exit the function

        # Only proceed if valid data is found
        price_data = pd.DataFrame([[date, price]], columns=['Date', 'Commodity'])
        print(price_data)

        upload_to_google_sheet(price_data, sheet_key, worksheet_name, row)

if __name__ == "__main__":
    main('1Qonj5yKwHVrxApUi7_N2CJtxj61rPfULXALrY4f8lPE', 'ALL_EU_AEA')

        Date             Commodity
0  30-May-25  [1630, 1657.5, 1685]
Row successfully added to ICIS_PE HDPE Blow Moulding EU at row 16.
        Date             Commodity
0  30-May-25  [1570, 1607.5, 1645]
Row successfully added to ICIS_PE HDPE Injection EU at row 16.
        Date             Commodity
0  30-May-25  [1395, 1440.0, 1485]
Row successfully added to ICIS_PP Homopolymer Injection EU at row 14.
        Date             Commodity
0  30-May-25  [1465, 1540.0, 1615]
Row successfully added to ICIS_PP Co-Polymer EU at row 14.


TimeoutException: Message: 
Stacktrace:
#0 0x5d24f9f02c4a <unknown>
#1 0x5d24f99a86e0 <unknown>
#2 0x5d24f99fa117 <unknown>
#3 0x5d24f99fa311 <unknown>
#4 0x5d24f9a48ec4 <unknown>
#5 0x5d24f9a1fe5d <unknown>
#6 0x5d24f9a462cc <unknown>
#7 0x5d24f9a1fc03 <unknown>
#8 0x5d24f99ec47b <unknown>
#9 0x5d24f99ed0e1 <unknown>
#10 0x5d24f9ec73fb <unknown>
#11 0x5d24f9ecb32f <unknown>
#12 0x5d24f9eaef39 <unknown>
#13 0x5d24f9ecbec8 <unknown>
#14 0x5d24f9e9368f <unknown>
#15 0x5d24f9ef02b8 <unknown>
#16 0x5d24f9ef0496 <unknown>
#17 0x5d24f9f01b26 <unknown>
#18 0x7cf2f377cac3 <unknown>


In [12]:
!pip install --upgrade PyMuPDF==1.24.9
!pip install --upgrade fitz==1.16.14
!pip install pygsheets
!pip install google-colab-selenium
!pip install retrying
import base64
import io
import time
import pandas as pd
import requests
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
# import fitz
import pymupdf
# from pdf2image import convert_from_path
import pygsheets
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default
import gspread
import re
import json
# Authenticate user
auth.authenticate_user()
creds, _ = default()



def pdf_to_img(pdf_path, img_path):
    # Open the PDF file
    pdf_document = pymupdf.open(pdf_path)

    # Get the first page
    page = pdf_document[0]

    # Render page to an image
    pixmap = page.get_pixmap(dpi=300)  # Adjust DPI as necessary

    # Save the image
    pixmap.save(img_path)

# def pdf_to_img(pdf_path, img_path):
#     # Convert PDF to image and save the first page as an image
#     images = convert_from_path(pdf_path, dpi=300)
#     images[0].save(img_path, 'JPEG')

def save_as_jpg(driver, path):
    screenshot = driver.get_screenshot_as_png()
    image = Image.open(io.BytesIO(screenshot))
    image = image.convert('RGB')
    image.save(path, 'JPEG')

def save_pdf(driver, path):
    settings = {
        "landscape": False,
        "displayHeaderFooter": False,
        "printBackground": True,
        "preferCSSPageSize": True
    }
    result = driver.execute_cdp_cmd("Page.printToPDF", settings)
    pdf_data = base64.b64decode(result['data'])
    with open(path, 'wb') as f:
        f.write(pdf_data)
    time.sleep(3)
    pdf_to_img('/content/webpage.pdf', '/content/01.jpg')

def fetch_RISI_data(link, search_term):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)
    driver.get(link)

    wait = WebDriverWait(driver, 100)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#userEmail')))

    driver.execute_script(f'document.querySelector("#userEmail").value = "beata_zbrozek@colpal.com"')
    driver.execute_script(f'document.querySelector("#password").value = "Risi2025!"')
    driver.execute_script(f'document.querySelector("#login-button").click()')
    time.sleep(5)

    # Dynamic search term
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mat-mdc-chip-list-input-0')))
    input_element = driver.execute_script("return document.querySelector('#mat-mdc-chip-list-input-0')")

    input_element.clear()
    input_element.send_keys(search_term, Keys.ENTER)

    time.sleep(5)
    driver.execute_script(f'document.querySelector("body > fui-root-container > fui-root > fui-dashboard-container > fui-dashboard > main > header > fui-workspace-header-container > fui-dashboard-header-container > fui-dashboard-header > div.header-section.search.draggable > fui-search-bar > fui-dropdown-container > div > div.content > div > fui-global-search-dropdown > div > section.results-view.ng-star-inserted > div > fui-search-feed > ul > cdk-virtual-scroll-viewport > div.cdk-virtual-scroll-content-wrapper > li > fui-instrument-search-result-item > fui-search-result-item").click()')
    time.sleep(5)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#mat-tab-label-1-3")))
    driver.execute_script(f'document.querySelector("#mat-tab-label-1-3").click()')
    time.sleep(5)
    save_pdf(driver, '/content/webpage.pdf')
    driver.close()

    return driver

def update_google_sheet_from_csv(csv_path, sheet_id, sheet_name):
    # Load CSV with correct headers setup
    df = pd.read_csv(csv_path)

    df.columns = df.iloc[0]  # Promote first row to header
    df = df[1:]  # Remove the header from the data
    df.reset_index(drop=True, inplace=True)

    df[df.columns[0]] = pd.to_datetime(df[df.columns[0]], errors='coerce')
    df = df.dropna(subset=[df.columns[0]])

    # Convert dates explicitly to string in the desired format
    df[df.columns[0]] = df[df.columns[0]].dt.strftime('%Y-%m-%d')

    df = df.sort_values(by=df.columns[0])

    client = gspread.authorize(creds)
    sh = client.open_by_key(sheet_id)
    worksheet = sh.worksheet(sheet_name)

    existing_data = pd.DataFrame(worksheet.get_all_values())

    if not existing_data.empty:
        existing_data.columns = existing_data.iloc[0]
        existing_data = existing_data[1:]
        existing_data.reset_index(drop=True, inplace=True)

    new_rows = df

    if not existing_data.empty and df.columns[0] in existing_data:
        records_to_add = new_rows[
            ~new_rows[df.columns[0]].isin(existing_data[existing_data.columns[0]].astype('str'))]
    else:
        records_to_add = new_rows

    if not records_to_add.empty:
        set_with_dataframe(worksheet, records_to_add, include_index=False,
                           include_column_header=False, row=len(existing_data) + 2)
        print("New data appended to Google Sheets successfully.")
    else:
        print("No new data to append.")

class ImageService:
    def __init__(self, api_key):
        self.api_key = api_key

    def get_image_base64_string(self, image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    def send_post_request(self, url, json_content):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        response = requests.post(url, data=json_content, headers=headers)
        if 200 <= response.status_code < 300:
            return response.text
        else:
            raise Exception(f"Request failed: {response.status_code}, {response.text}")

    # def get_results(self, image_path):
    #     url = "https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions"
    #     encoded_image = self.get_image_base64_string(image_path)
    #     json_content = {
    #         "model": "qwen2-vl-7b-instruct",
    #         "messages": [
    #             {
    #                 "role": "user",
    #                 "content": [
    #                     {"type": "text", "text": "抓取图片中表格。"},
    #                     {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}}
    #                 ]
    #             }
    #         ]
    #     }
    #     json_content_str = json.dumps(json_content)
    #     result = self.send_post_request(url, json_content_str)
    #     return result

    def get_results(self, image_path):
        url = "https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions"
        encoded_image = self.get_image_base64_string(image_path)
        json_content = {
            "model": "qwen-vl-plus",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "抓取图片中表格。"},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}}
                    ]
                }
            ]
        }
        json_content_str = json.dumps(json_content)
        result = self.send_post_request(url, json_content_str)
        return result

    def parse_table_and_save_to_csv(self, response_body, csv_path):
        data = json.loads(response_body)
        if 'choices' in data and len(data['choices']) > 0:
            message = data['choices'][0]['message']['content']
            table_lines = re.findall(r"\|.*\|", message)

            if table_lines:
                rows = []
                for line in table_lines[1:]:
                    columns = [col.strip() for col in line.split('|')[1:-1]]
                    rows.append(columns)

                header = [col.strip() for col in table_lines[0].split('|')[1:-1]]
                table_df = pd.DataFrame(rows, columns=header)
                table_df.to_csv(csv_path, index=False)
                print(f"Extracted table saved to {csv_path}")
            else:
                print("No table data found in response.")
        else:
            print("No table data found in response.")

def get_dynamic_parameters(sheet_id):
    client = gspread.authorize(creds)
    sh = client.open_by_key(sheet_id)
    worksheet = sh.worksheet("ALL_EU_AEA")

    data = worksheet.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])

    filtered_df = df[df['Tab Name'].str.startswith("RISI")]

    return filtered_df[['Address', 'Tab Name', 'Series code']].to_dict('records')

def risi_main():
    sheet_id = '1Qonj5yKwHVrxApUi7_N2CJtxj61rPfULXALrY4f8lPE'

    rows = get_dynamic_parameters(sheet_id)

    api_key = "sk-db6fa6efbf144521b0180da5e83b0a94"
    image_path = '/content/01.jpg'

    service = ImageService(api_key)

    for row in rows:
        link = row['Address']
        sheet_name = row['Tab Name']
        search_term = row['Series code']  # Dynamic search term from column E

        # Fetch data from the website using the dynamic search term
        fetch_RISI_data(link, search_term)
        csv_path = '/content/extracted_table.csv'
        try:
            result = service.get_results(image_path)
            print(result)
            service.parse_table_and_save_to_csv(result, csv_path)
        except Exception as e:
            print(f"An error occurred: {e}")

        # Update Google Sheet
        update_google_sheet_from_csv(csv_path, sheet_id, sheet_name)
risi_main()

ERROR: Could not find a version that satisfies the requirement fitz==1.16.14 (from versions: 0.0.1.dev0, 0.0.1.dev2)
ERROR: No matching distribution found for fitz==1.16.14
{"choices":[{"message":{"content":"以下是您提供的图像中的表格内容：\n\n| Date         | Mid |\n|--------------|-----|\n| 01 May 2025   | 727,50    |\n| 01 Apr 2025   | 687,50     |\n| 01 Mar 2025   | 687,50      |\n| 01 Feb 2025   | 687,50       |\n| 01 Jan 2025   | 607,50        |\n| 01 Dec 2024   | 627,50          |\n| 01 Nov 2024   | 647,50           |\n| 01 Oct 2024   | 667,50            |\n| 01 Sep 2024   | 707,50             |\n\n如果您需要进一步处理或分析这些数据，请告诉我！","role":"assistant"},"finish_reason":"stop","index":0,"logprobs":null}],"object":"chat.completion","usage":{"prompt_tokens":1256,"completion_tokens":220,"total_tokens":1476,"prompt_tokens_details":{"cached_tokens":0}},"created":1748914632,"system_fingerprint":null,"model":"qwen-vl-plus","id":"chatcmpl-e5c4df4c-02f5-9c1a-b980-0baee27929fe"}
Extracted table saved to /content/ext